In [ ]:
class Node:
    def __init__(self, count):
        self.count = count
        self.keys = set()
        self.prev = None
        self.next = None

class AllOne:
    def __init__(self):
        self.key_count_map = {}  # Maps keys to their count
        self.count_bucket_map = {}  # Maps count to the corresponding bucket (Node)
        self.head = Node(float('-inf'))  # Dummy head for min bucket
        self.tail = Node(float('inf'))   # Dummy tail for max bucket
        self.head.next = self.tail
        self.tail.prev = self.head

    def _remove_node(self, node):
        """ Remove the node from the doubly linked list. """
        prev_node = node.prev
        next_node = node.next
        prev_node.next = next_node
        next_node.prev = prev_node
        del self.count_bucket_map[node.count]

    def _add_node_after(self, new_node, prev_node):
        """ Add new_node after prev_node in the doubly linked list. """
        next_node = prev_node.next
        prev_node.next = new_node
        new_node.prev = prev_node
        new_node.next = next_node
        next_node.prev = new_node
        self.count_bucket_map[new_node.count] = new_node

    def inc(self, key):
        if key in self.key_count_map:
            current_count = self.key_count_map[key]
            current_node = self.count_bucket_map[current_count]
            new_count = current_count + 1
            self.key_count_map[key] = new_count

            # Move the key to the new count bucket
            current_node.keys.remove(key)
            if new_count not in self.count_bucket_map:
                new_node = Node(new_count)
                self._add_node_after(new_node, current_node)
            self.count_bucket_map[new_count].keys.add(key)

            # If the old bucket is empty, remove it
            if not current_node.keys:
                self._remove_node(current_node)
        else:
            # First time the key is added
            self.key_count_map[key] = 1
            if 1 not in self.count_bucket_map:
                new_node = Node(1)
                self._add_node_after(new_node, self.head)
            self.count_bucket_map[1].keys.add(key)

    def dec(self, key):
        current_count = self.key_count_map[key]
        current_node = self.count_bucket_map[current_count]
        new_count = current_count - 1

        # Move the key to the new count bucket or delete if count is 0
        current_node.keys.remove(key)
        if new_count == 0:
            del self.key_count_map[key]  # Remove key entirely if count is zero
        else:
            self.key_count_map[key] = new_count
            if new_count not in self.count_bucket_map:
                new_node = Node(new_count)
                self._add_node_after(new_node, current_node.prev)
            self.count_bucket_map[new_count].keys.add(key)

        # If the old bucket is empty, remove it
        if not current_node.keys:
            self._remove_node(current_node)

    def getMaxKey(self):
        if self.tail.prev == self.head:
            return ""
        return next(iter(self.tail.prev.keys))

    def getMinKey(self):
        if self.head.next == self.tail:
            return ""
        return next(iter(self.head.next.keys))
